In [40]:
import spotipy
import base64
import requests
from spotipy.oauth2 import SpotifyClientCredentials
from creds import CLIENT_ID, CLIENT_SECRET
import pandas as pd
import time

In [41]:
# token
auth_manager = SpotifyClientCredentials(CLIENT_ID, CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)


In [42]:
#Get spotify playlist ids and names
def get_all_playlists(user_id='spotify'): #api call
    playlist_ids = []
    playlist_names = []
    playlists = sp.user_playlists(user_id) #api call
    #my_playlists = sp.user_playlist('z8ctw1rqti6nguka70zk1ishf')

    while playlists: # example provided in spotipy documentation
        for i, playlist in enumerate(playlists['items']):
            #print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
            playlist_ids.append(playlist['uri'][-22:])
            playlist_names.append(playlist['name'])
        if playlists['next']:
            playlists = sp.next(playlists)
            
        else:
            playlists = None
    return playlist_ids, playlist_names


In [45]:
# Get track info from playlists into a dataframe - separated out API calls

def get_playlist_tracks(playlist_id):#api call
    try: 
        tracks = sp.playlist_tracks(playlist_id, limit = 100, fields='items(track(id, name, artists, album(id, name)))')
        return tracks
    
    except Exception as e:
        print(f"get_playlist_tracks error fetching {e}")
        return []

# def get_track_details(track_id):
#     try:
#         track_info = sp.track(track_id) if track_id else None
#         return track_info['popularity']
#     except Exception as e:
#         print(f"Error fetching track {e}")
#         return None

def get_audio_features(track_id): #api call
    try:
        audio_stuffs = sp.audio_features(track_id)[0] if track_id else None 
        return audio_stuffs
    except Exception as e:
        print(f"get_audio_features error fetching {e}")
        return None
    
def get_tracks(playlist_id, playlist_name):
    music_data = []
    tracks = get_playlist_tracks(playlist_id)  

    for track_info in tracks['items']:
        track = track_info['track']

        if track:
            track_name = track['name']
            artists = ', '.join([artist['name'] for artist in track['artists']])
            album_name = track['album']['name']
            album_id = track['album']['id']
            track_id = track['id']
        
        audio_stuffs = get_audio_features(track_id) #api call
        # popularity = get_track_details(track_id) #api call


        track_data = {
            'Playlist Name': playlist_name,
            'Track': track,
            'Track Name': track_name,
            'Artist': artists,
            'Album Name': album_name,
            'Playlist ID': playlist_id,
            'Album ID': album_id,
            'Track ID': track_id,
            # 'Popularity': popularity,
            'Acousticness': audio_stuffs['acousticness'] if audio_stuffs else None,
            'Danceability': audio_stuffs['danceability'] if audio_stuffs else None,
            'Energy': audio_stuffs['energy'] if audio_stuffs else None,
            'Instrumentalness': audio_stuffs['instrumentalness'] if audio_stuffs else None,
            'Loudness': audio_stuffs['loudness'] if audio_stuffs else None,
            'Speechiness': audio_stuffs['speechiness'] if audio_stuffs else None,
            'Tempo': audio_stuffs['tempo'] if audio_stuffs else None,
            'Mode': audio_stuffs['mode'] if audio_stuffs else None,
            'Valence': audio_stuffs['valence'] if audio_stuffs else None
        }

        music_data.append(track_data)
    return music_data

all_tracks = []  
playlist_ids, playlist_names = get_all_playlists()

for playlist_id, playlist_name in zip(playlist_ids, playlist_names): #Test with first 10 IDs
    #print(f"Getting tracks from playlist ID: {id}")
    tracks = get_tracks(playlist_id,playlist_name) #Two API calls per function call
    all_tracks.extend(tracks)
    time.sleep(5) # to keep the spotify api server from giving me a 429 error when they finally take me back
    break # break after one until code is fixed

spotify_track_df = pd.DataFrame(all_tracks, columns=['Playlist Name','Track Name', 'Artist', 'Album Name', 'Playlist ID','Album ID', 'Track ID',
                                                     'Acousticness', 'Danceability','Energy', 'Instrumentalness', 
                                                     'Loudness', 'Speechiness', 'Tempo', 'Mode', 'Valence'])

spotify_track_df.head()


,Playlist Name,Track Name,Artist,Album Name,Playlist ID,Album ID,Track ID,Acousticness,Danceability,Energy,Instrumentalness,Loudness,Speechiness,Tempo,Mode,Valence
0,Today’s Top Hits,"Good Luck, Babe!",Chappell Roan,"Good Luck, Babe!",37i9dQZF1DXcBWIGoYBM5M,1WAjjRMfZjEXtB0lQrAw6Q,0WbMK4wrZ1wFSty9F7FCgu,0.05020,0.700,0.582,0.000000,-5.960,0.0356,116.712,0,0.785
1,Today’s Top Hits,Please Please Please,Sabrina Carpenter,Please Please Please,37i9dQZF1DXcBWIGoYBM5M,5bBaoign62r1i7OV8w7mi9,5N3hjp1WNayUPZrA8kJmJP,0.27400,0.669,0.586,0.000000,-6.073,0.0540,107.071,1,0.579
2,Today’s Top Hits,I Had Some Help (Feat. Morgan Wallen),"Post Malone, Morgan Wallen",I Had Some Help,37i9dQZF1DXcBWIGoYBM5M,1woYXxyyxTQJ0E0AhZE6mj,7221xIgOnuakPdLqT0F3nP,0.00757,0.638,0.855,0.000000,-4.860,0.0264,127.986,1,0.731
3,Today’s Top Hits,BIRDS OF A FEATHER,Billie Eilish,HIT ME HARD AND SOFT,37i9dQZF1DXcBWIGoYBM5M,7aJuG4TFXa2hmE4z1yxc3n,6dOtVTDdiauQNBQEDOtlAB,0.20000,0.747,0.507,0.060800,-10.171,0.0358,104.978,1,0.438
4,Today’s Top Hits,Too Sweet,Hozier,Unheard,37i9dQZF1DXcBWIGoYBM5M,1KZC0cX0qq6hodR9YVgh9F,4IadxL6BUymXlh8RCJJu7T,0.02950,0.741,0.620,0.000809,-5.505,0.0412,117.038,1,0.934
